In [1]:
import numpy as np
import time
import motors

In [23]:
class SpringFollower:
    def __init__(self, speed=0.5, k=10.0, b=6.0, rest_length=0.5, dt=0.02):
        self.speed = speed
        self.motors = motors.MotorsYukon(mecanum=False)
                                         
        self.k = k
        self.b = b
        self.l0 = rest_length
        self.dt = dt

        self.vel = np.array([0.0, 0.0])
        self._origin = np.array([0.0, 0.0])

    def human_position(self, r, theta):
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        return np.array([x, y])

    def update(self, human_position):
        distance = human_position[0]
        angle = human_position[1]
        
        ph = self.human_position(human_position[0], human_position[1])
        diff = ph - self._origin
        dist = np.linalg.norm(diff)

        if dist > 1e-6:
            vel_spring = (self.k / self.b) * (1 - self.l0 / dist) * diff
        else:
            vel_spring = np.zeros(2)

        self.vel = vel_spring / self.b
        self.pos += self.vel * self.dt
        return self.vel

    def drive(self, angle, speed):
        # max angle is 90 degrees
        # at -90 and 90 degrees wheel multiplier should be 0 for corresponding side
        # as robot is turning theta decreeses
        temp = angle
        np.clip(temp, 1, 1)
        
        turning = np.cos(angle)
        
        print("Turning: ", turning)
        if angle > 0:
            print("turning right")
            self.motors.frontLeft(speed=self.speed)
            self.motors.backLeft(speed=self.speed)

            turning_speed = turning*self.speed
            self.motors.frontRight(speed=turning_speed)
            self.motors.backRight(speed=turning_speed)

        elif angle < 0:
            
            print("turning left")

            self.motors.frontLeft(speed=turning_speed)
            self.motors.backLeft(speed=turning_speed)

            turning_speed = turning*self.speed
            self.motors.frontRight(speed=self.speed)
            self.motors.backRight(speed=self.speed)

        else:
            self.motors.frontLeft(speed=self.speed)
            self.motors.backLeft(speed=self.speed)

            turning_speed = turning*self.speed
            self.motors.frontRight(speed=self.speed)
            self.motors.backRight(speed=self.speed)


    def move(self, vel):
        theta = np.angle(vel)

        #while 
        #if theta > 0:
            
        
        

In [24]:
follower = SpringFollower()

In [28]:
angle = np.pi/2
follower.drive(angle, 0)

time.sleep(5)

follower.motors.stop()

Turning:  6.123233995736766e-17
turning right


In [ ]:
while True:
    # get sensor information
    human_position = get_position() # vector of (distance, angle)
    
    # decide heading
    vel = robot.update(human_position) # vector of (vx, vy)
    
    # tell the motors
    robot.move(vel)